In [2]:
import json
import tkinter as tk
from tkinter import messagebox, filedialog, StringVar
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import networkx as nx

from antlr4 import *
from XMLLexer import XMLLexer
from XMLParser import XMLParser
from parse_xml import DAO_ML_Visitor
from translator import *

graphs = []
visitor = None

def configure_translator():
    config_window = tk.Toplevel(root)
    config_window.title("Configure Translator")
    config_window.geometry("400x300")
    config_window.configure(bg="#f5f5f5")

    logic_label = tk.Label(config_window, text="Select Translation Logic:", font=("Helvetica", 12), bg="#f5f5f5")
    logic_label.pack(pady=10)

    simple_radio = tk.Radiobutton(config_window, text="Simple", variable=translation_logic, value="simple", font=("Helvetica", 12), bg="#f5f5f5")
    simple_radio.pack(pady=5)

    optimized_radio = tk.Radiobutton(config_window, text="Optimized", variable=translation_logic, value="optimized", font=("Helvetica", 12), bg="#f5f5f5")
    optimized_radio.pack(pady=5)

    save_button = tk.Button(config_window, text="Save Configuration", font=("Helvetica", 12), bg="#4CAF50", fg="white", command=config_window.destroy)
    save_button.pack(pady=20)


def translate():
    xml_file = file_entry.get()

    if not xml_file:
        messagebox.showerror("Error", "Please enter the XML file name.")
        return None

    try:
        input_stream = FileStream(xml_file)
        lexer = XMLLexer(input_stream)
        stream = CommonTokenStream(lexer)
        parser = XMLParser(stream)
        tree = parser.document()

        visitor = DAO_ML_Visitor()
        visitor.visit(tree)

        dao_id_list = []
        for dao_id in visitor.daos.keys():
            dao_id_list.append(dao_id)
            dao = visitor.daos[dao_id]
            # the translator uses the selected translation logic
            translator = SolidityTranslator(dao, translation_logic.get())  
            solidity_code = translator.translate()

            filename = dao.dao_name + ".sol"
            full_path = f'{filename}'
            with open(full_path, 'w') as f:
                f.write(solidity_code)
                f.flush()

            control_graph = visitor.get_control_graph()
            graphs.append(control_graph)
            
        dao_ids = ", ".join(dao_id_list) + ".sol"

        if graphs:
            display_button.configure(bg="green")
            print("CONTROL GRAPH IS at the end of translate", graphs)
        else:
            display_button.configure(bg="gray")
            print("CONTROL GRAPH IS at the end of translate", graphs)

        messagebox.showinfo("Success", f"Translation completed successfully! {dao_ids} created.")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")
        print("Error", f"An error occurred: {e}")


def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("XML files", "*.xml")])
    file_entry.delete(0, tk.END)
    file_entry.insert(0, file_path)


def display_control_graph():
    print("CONTROL GRAPH IS at the beginning of display control graphs", graphs)

    if not graphs:
        messagebox.showinfo("Info", "No control graph to display.")
        return

    graph_window = tk.Toplevel(root)
    graph_window.title("Control Graph")
    graph_window.geometry("1000x700")

    fig, ax = plt.subplots(figsize=(20, 20))
    
    for graph in graphs:
        pos = nx.spring_layout(graph)
        nx.draw(graph, 
                pos, 
                with_labels=True, 
                node_color="blue", 
                node_size=1000,  
                ax=ax, 
                font_size=10,  
                edge_color="black",
                linewidths=1, 
                width=2)

    canvas = FigureCanvasTkAgg(fig, master=graph_window)
    canvas.draw()
    canvas.get_tk_widget().pack()


def print_object_properties():
    global visitor
    if not visitor:
        print("No object properties to display.")
        return

    i = 0
    for dao in visitor.daos.values():
        filename = "dao_properties_" + str(i) + ".txt"
        with open(filename, "w") as file:
            file.write(f"\n--- DAO {i + 1} ---\n")
            file.write(str(dao))
        i += 1
    
    messagebox.showinfo("Success", "DAO properties have been saved to dao_properties.txt")


def print_json():
    for dao in visitor.daos.values():
        filename = "dao_" + dao.dao_name + ".json"
        with open(filename, "w") as f:
            json.dump(dao, f)
    messagebox.showinfo("Success", "DAO properties have been saved to JSON files")


root = tk.Tk()
root.title("DAO-ML to Solidity Translator")
root.geometry("1200x1000")
root.configure(bg="#f5f5f5")

# Default to "simple" logic
translation_logic = StringVar(value="simple")  

title_label = tk.Label(root, text="DAO-ML to Solidity Translator", font=("Helvetica", 16, "bold"), bg="#f5f5f5", fg="#333")
title_label.pack(pady=10)

frame = tk.Frame(root, bg="#f5f5f5")
frame.pack(pady=10)

file_label = tk.Label(frame, text="XML File:", font=("Helvetica", 12), bg="#f5f5f5", fg="#333")
file_label.pack(side=tk.LEFT, padx=5)

file_entry = tk.Entry(frame, width=40, font=("Helvetica", 12))
file_entry.pack(side=tk.LEFT, padx=5)

browse_button = tk.Button(frame, text="Browse", font=("Helvetica", 12), command=browse_file)
browse_button.pack(side=tk.LEFT, padx=5)

translate_button = tk.Button(root, text="Translate DAO-ML Model to Solidity", font=("Helvetica", 14), bg="#4CAF50", fg="white", command=translate)
translate_button.pack(pady=20)

config_button = tk.Button(root, text="Configure Translator", font=("Helvetica", 14), bg="#FF9800", fg="white", command=configure_translator)
config_button.pack(pady=20)

display_button = tk.Button(root, text="Display Control Graph", font=("Helvetica", 14), bg="gray", fg="white", command=display_control_graph)
display_button.pack(pady=20)

print_button = tk.Button(root, text="Print Object Properties", font=("Helvetica", 14), bg="#4CAF50", fg="white", command=print_object_properties)
print_button.pack(pady=20)

print_button = tk.Button(root, text="Print to JSON", font=("Helvetica", 14), bg="#4CAF50", fg="white", command=print_json)
print_button.pack(pady=20)

root.mainloop()


DAO created with ID: GCDAO
Role created with ID: GroupMember
Role GroupMember is associated with propose_service_provision
Role GroupMember is associated with update_user_profile
Role GroupMember is associated with funding_request_submission
Role GroupMember is associated with ServiceProvision
Role created with ID: TreasuryManager
Role TreasuryManager is associated with funding_request_assessment
Role TreasuryManager is associated with set_limits_to_group_currency_minting
Role TreasuryManager is associated with TreasuryManagement
Role TreasuryManager is controlled byEconomicCouncil
Role or Committee TreasuryManager aggregates into Economic
Role or Committee TreasuryManager aggregates into ActiveMember
Role created with ID: ActiveMember
Role or Committee ActiveMember aggregates into CommunityCouncil
Role or Committee ActiveMember aggregates into EconomicCouncil
Role or Committee ActiveMember aggregates into TechnicalCouncil
Role or Committee ActiveMember aggregates into GroupMember
Comm